In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Basic Libraries

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os


# Defining the Needed Properties for Preprocessing

In [ ]:
# Define Constants
FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

# Loading Train Data


In [ ]:
filenames = os.listdir("/content/drive/MyDrive/AutismDataset(1)/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'Autistic':
        categories.append(1)
    else:
        categories.append(0)

train_df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
train_df.head()


In [ ]:
train_df.tail()


# Plotting Train Data

In [ ]:
train_df['category'].value_counts().plot.bar()


In [ ]:
test_filenames = os.listdir("/content/drive/MyDrive/AutismDataset(1)/test")
categories = []
for filename in test_filenames:
    category = filename.split('.')[0]
    if category == 'Autistic':
        categories.append(1)
    else:
        categories.append(0)

test_df = pd.DataFrame({
    'filename': test_filenames,
    'category': categories
})

In [ ]:
test_df.head()


In [ ]:
test_df.tail()

# Plotting Test Data

In [ ]:
test_df['category'].value_counts().plot.bar()

# Loading Random Images From The Dataset


In [ ]:
# See Sample Image
sample = random.choice(filenames)
image = load_img("/content/drive/MyDrive/AutismDataset(1)/train/"+sample)
plt.imshow(image)

In [ ]:
base_path = "/content/drive/MyDrive/AutismDataset(1)/valid"
subfolders = ['Autistic', 'Non_Autistic']
valid_filenames = []
categories = []

for subfolder in subfolders:
    folder_path = os.path.join(base_path, subfolder)
    filenames = os.listdir(folder_path)
    valid_filenames.extend([os.path.join(subfolder, filename) for filename in filenames])
    if subfolder == 'autistic':
        categories.extend([1] * len(filenames))
    else:  # Assuming any other folder would be 'non_autistic'
        categories.extend([0] * len(filenames))

valid_df = pd.DataFrame({
    'filename': valid_filenames,
    'category': categories
})

print(valid_df.head())  # Display the first few rows of the DataFrame to verify


In [ ]:
valid_df.head()


In [ ]:
valid_df.tail()

In [ ]:
valid_df['category'].value_counts().plot.bar()

In [ ]:
train_df["category"] = train_df["category"].replace({0: 'Non_Autistic', 1: 'Autistic'})

In [ ]:
test_df["category"] = test_df["category"].replace({0: 'Non_Autistic', 1: 'Autistic'})

In [ ]:
valid_df["category"] = valid_df["category"].replace({0: 'Non_Autistic', 1: 'Autistic'})

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
test_df['category'].value_counts().plot.bar()

In [ ]:
valid_df['category'].value_counts().plot.bar()

# Building The Model

In [ ]:
# Build Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                             patience=2,
                                             verbose=1,
                                             factor=0.5,
                                             min_lr=0.00001)
callbacks = [earlystop, learning_rate_reduction]

# Preparing the data

# Spliting the Train and Test Data

In [ ]:
train_df, validate_df = train_test_split(train_df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

# Creating Training Generator For Train Images

In [ ]:
# Training Generator
train_datagen = ImageDataGenerator(
    rotation_range=10,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    "/content/drive/MyDrive/AutismDataset(1)/train",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

# Creating Validation Generator For Test Images

In [ ]:
# Validation Generator
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    "/content/drive/MyDrive/AutismDataset(1)/train",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

# Fitting The Model

In [ ]:
#Fit Model
epochs=40
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
)

# Accuracy and Loss on Test Data

In [ ]:
loss, accuracy = model.evaluate_generator(validation_generator, total_validate//batch_size, workers=12)
print("Test: accuracy = %f  ;  loss = %f " % (accuracy, loss))

# Saving The Model

In [ ]:
model.save("CNN.h5")


# Ploting the Loss And Accuracy

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))


legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

# Preparing Testing Data

In [ ]:
test_filenames = os.listdir("/content/drive/MyDrive/AutismDataset(1)/test")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

# Creating Testing Generator

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df,
    "/content/drive/MyDrive/AutismDataset(1)/test",
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

# Predicting on Test Data


In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
predict

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)
test_df['category']

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category']

In [ ]:
test_df['category'] = test_df['category'].replace({ 'Autistic': 1, 'Non_Autistic': 0 })
test_df['category']

# Virtaulize Result


In [ ]:
test_df['category'].value_counts().plot.bar()

# See The Predicted Result on Test Data

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("/content/drive/MyDrive/AutismDataset(1)/test/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()

# Creating Testing Images Outside the Dataset to Predict The Result

In [ ]:
base_directory = "/content/drive/MyDrive/AutismDataset(1)/valid"
all_filenames = []

# Iterate through each subfolder
for category in os.listdir(base_directory):
    category_directory = os.path.join(base_directory, category)
    # Iterate through each file in the subfolder
    for filename in os.listdir(category_directory):
        # Append filename with subfolder information to the list
        all_filenames.append(os.path.join(category, filename))

# Create DataFrame from the list of filenames
test_df2 = pd.DataFrame({'filename': all_filenames})

# Get the number of samples in the DataFrame
nb_samples = test_df2.shape[0]

print("Number of samples:", nb_samples)


# Creating Test Generator  for the Images To Predict

In [ ]:
test_gen2 = ImageDataGenerator(rescale=1./255)
test_generator2 = test_gen2.flow_from_dataframe(
    test_df2,
    "/content/drive/MyDrive/AutismDataset(1)/valid",
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

# Predictions On The Images

In [ ]:
predict2 = model.predict_generator(test_generator2, steps=np.ceil(nb_samples/batch_size))
predict2

# Printing The Results

In [ ]:
test_df2['category'] = np.argmax(predict2, axis=-1)
test_df2['category']

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df2['category'] = test_df2['category'].replace(label_map)
test_df2['category']

In [ ]:
from keras.preprocessing import image
from keras.models import load_model
import numpy as np

# Load the saved model
model = load_model("/content/CNN.h5")

# Define the target image path
image_path = "/content/drive/MyDrive/AutismDataset(1)/test/Non_Autistic.1.jpg"

# Load and preprocess the image
img = image.load_img(image_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image

# Predict using the loaded model
prediction = model.predict(img_array)
print(prediction)
# Interpret the prediction
if prediction[0][0] < prediction[0][1]:
    print("The image is predicted to be Non-Autistic.")
else:
    print("The image is predicted to be Autistic.")
